In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [2]:
# Defining Butterworth Low Pass Filter and Normalizing functions

def butter_lowpass_filter(window, cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, window)
    return y


def NormalizeData(window):
    return (window - np.min(window)) / (np.max(window) - np.min(window))

In [3]:
df = pd.read_csv('1001_H.csv',sep=";", header=None)

In [4]:
df = df[(df[1] == 0) | (df[1] == 1)]

In [5]:
r_peak = df[0].values

In [6]:
v1_v5 = pd.read_csv('1001.csv', header= None)

In [7]:
v1 = v1_v5[0].values

In [8]:
# Creating windows and storing in lists 

x = 1024
total_values = 1350000
loop_count = total_values // x
r_peak_package = []
sig_package = []

for r in range(loop_count):
    start_index = r*x
    end_index = r*x + x
    sig = v1[start_index : end_index]
    r_index = r_peak[(r_peak > start_index) & (r_peak < end_index)]
    r_peak_package.append(r_index)
    sig_package.append(sig)
    
loop_count, len(r_peak_package)

(1318, 1318)

In [9]:
r_peak_list = []


for i in range(len(r_peak_package)):
    start_index = i * 1024
    r_peak_p = np.array(r_peak_package[i])
    r_peak_p = r_peak_p - start_index
    r_peak_list.append(r_peak_p)
    # plt.plot(sig_package[i])
    # plt.scatter(r_peak_list[i], sig_package[i][r_peak_list[i]])
    # plt.title(f"#{i+1}")


In [10]:
# Creating 512 index sized and 20 index inersected windows 

signal = np.array(v1)
window_size = 512
intersection = 20

window = [signal[i:i+window_size] 
          for i in range(0, len(signal) - window_size + 1, window_size - intersection)]

In [11]:
# Applying butter filter and normalizing functions for lists

r_peak_list_new = []
signal_list_new = []
cutoff_freq = 25
sampling_rate = 125
r_peak_array = np.array(r_peak)
loop_count = len(v1) // 512
for i in range(len(v1)):
    if i == 0:
        start_index = i*512
        end_index = i*512 + 512
    else:
        start_index = end_index - 20
        end_index = start_index + 512
    
    if end_index > len(v1):
        break
        
    sig = v1[start_index:end_index]
    filtered_sig = butter_lowpass_filter(sig, cutoff_freq, sampling_rate)
    filtered_sig = NormalizeData(filtered_sig)
    r_ = r_peak_array[(r_peak_array > start_index) & (r_peak_array < end_index)]
    r_ = r_ - start_index
    r_peak_list_new.append(r_)
    signal_list_new.append(filtered_sig)
    
    

In [12]:
# Creating Zero Vector with 1 value at peak points 10 indices long, starting and ending 7 indices out. 

vector_length = 512
vector = np.zeros(vector_length)

vector_list = []

for i in range(len(r_peak_list_new)):
    r_array = r_peak_list_new[i]

    vector = np.zeros(vector_length)
    
    for a in range(len(r_array)):
        r_ = r_array[a]
        if (r_ >= 7) and (r_ < 505):
            vector[r_ -5: r_ +5] = 1
    
    vector_list.append(vector)

In [13]:
'''

for i in range(len(signal_list_new)):
    plt.figure(figsize=(18,6))
    plt.plot(signal_list_new[i])
    plt.plot(vector_list[i])
    plt.scatter(r_peak_list_new[i], signal_list_new[i][r_peak_list_new[i]])
    plt.title(f"#{i+1}")
    
    '''

'\n\nfor i in range(len(signal_list_new)):\n    plt.figure(figsize=(18,6))\n    plt.plot(signal_list_new[i])\n    plt.plot(vector_list[i])\n    plt.scatter(r_peak_list_new[i], signal_list_new[i][r_peak_list_new[i]])\n    plt.title(f"#{i+1}")\n    \n    '

In [14]:
# Designing 1D U-Net model for training

def unet_1d(input_shape):
    inputs = layers.Input(shape=input_shape)

    c1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv1D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling1D(2)(c1)

    c2 = layers.Conv1D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv1D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling1D(2)(c2)

    c3 = layers.Conv1D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv1D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling1D(2)(c3)

    c4 = layers.Conv1D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv1D(512, 3, activation='relu', padding='same')(c4)



    u5 = layers.UpSampling1D(2)(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv1D(256, 3, activation='relu', padding='same')(u5)
    c5 = layers.Conv1D(256, 3, activation='relu', padding='same')(c5)

    u6 = layers.UpSampling1D(2)(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv1D(128, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv1D(128, 3, activation='relu', padding='same')(c6)

    u7 = layers.UpSampling1D(2)(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv1D(64, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv1D(64, 3, activation='relu', padding='same')(c7)

    outputs = layers.Conv1D(1, 1, activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model


In [15]:
model = unet_1d((512,1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 512, 64)      256         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 512, 64)      12352       ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 256, 64)      0           ['conv1d_1[0][0]']               
                                                                                              

In [16]:
signal_array = np.array(signal_list_new)
vector_array = np.array(vector_list)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(signal_array, vector_array, test_size=0.2, random_state=42)

In [18]:
input_shape = X_train.shape[1:]

model = unet_1d((512,1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
model = model.save_csv('model.h5')